# Radiation Integrals in the Swiss Light Source

Radiation Integrals:
$$I_1 = \oint \frac{\eta}{\rho} \text{d}s$$
$$I_2 = \oint \frac{1}{\rho^2} \text{d}s$$
$$I_3 = \oint \frac{1}{|\rho|^3} \text{d}s$$
$$I_4 = \oint \frac{1 - 2n}{|\rho|^3} \eta \text{d}s$$
$$I_5 = \oint \frac{\mathcal{H}}{|\rho|^3} \text{d}s$$

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import xtrack as xt
import xobjects as xo

from scipy.constants import c as clight

In [2]:
line = xt.Line.from_json('../Example_Data/b075_2024.09.25.json')
line.particle_ref = xt.Particles(q0 = -1, mass0 = xt.ELECTRON_MASS_EV, kinetic_energy0=2.7e9)

Loading line from dict:   0%|          | 0/4701 [00:00<?, ?it/s]

Done loading line from dict.           


In [14]:
# Frequency and voltage of the cavities from the technical design report.
cavfreq  = 499.6537e6
cavvolt  = 1780e3

# Create a table to inspect elements if needed.
tab = line.get_table()

# This is the place where arc 04 ends and long straight 05 starts.
# The cavities should be located somewhere here.
# The drift region is about 4 meters long, so somewhere between s_start_long05 and s_start_long05 + 4.
s_start_long05 = tab['s', 'ars05_gmrk_0000']
s_start_maincavity = s_start_long05 + 1

# Insert main cavities.
line.insert_element('maincav01', at_s=s_start_maincavity, element=xt.Cavity(frequency=cavfreq, voltage=cavvolt, lag=180))

In [15]:
slicing_strategies = [
    xt.Strategy(slicing=xt.Teapot(8)),  # Default
    xt.Strategy(slicing=xt.Teapot(2), element_type=xt.Bend),
    xt.Strategy(slicing=xt.Teapot(8), element_type=xt.Quadrupole),
    xt.Strategy(slicing=xt.Teapot(8), element_type=xt.Multipole),
]
line.slice_thick_elements(slicing_strategies)

# Build the tracker.
line.discard_tracker()
line.build_tracker(_context=xo.ContextCpu())
line.configure_radiation(model='mean')


# Tracking ------------------------------------------------------------------------------------------------------------------------------------------
# Calculate the Twiss parameters.
tw = line.twiss(method='6d', strengths=True, eneloss_and_damping=True, radiation_integrals=True)

Slicing line:   0%|          | 0/105281 [00:00<?, ?it/s]

In [11]:
tw._compute_radiation_integrals()

Table: 105281 rows, 17 cols
name                                 s        length rad_int_kappax rad_int_kappay    rad_int_hx ...
ring$start                           0             0             -0             -0   5.17113e-08
ars01_gmrk_0000                      0             0             -0             -0   5.17113e-08
drift_0                              0           0.4             -0             -0   5.17113e-08
drift_ars01_mkik_0050..1           0.4           0.3             -0             -0   5.17111e-08
ars01_mkik_0050                    0.7           0.6             -0             -0   5.17111e-08
drift_ars01_mkik_0050..2           0.7           0.3             -0             -0   5.17111e-08
drift_1                              1          0.47             -0             -0    5.1711e-08
drift_ars01_mkik_0140..1          1.47           0.3             -0             -0   5.17108e-08
ars01_mkik_0140                   1.77           0.6             -0             -0   5.17108e-0

In [12]:
print(f'Damping Constants:')
print(f'From Radiation Integrals:')
print(f'x: {tw.rad_int_damping_constant_x_s} [s⁻¹]')
print(f'y: {tw.rad_int_damping_constant_y_s} [s⁻¹]')
print(f'z: {tw.rad_int_damping_constant_zeta_s} [s⁻¹]\n')

print(f'From Twiss:')
print(f'x: {tw.damping_constants_s[0]} [s⁻¹]')
print(f'y: {tw.damping_constants_s[1]} [s⁻¹]')
print(f'z: {tw.damping_constants_s[2]} [s⁻¹]')

Damping Constants:
From Radiation Integrals:
x: 104.90162341243625 [s⁻¹]
y: 132.07332690382182 [s⁻¹]
z: 291.3183572990292 [s⁻¹]

From Twiss:
x: 241.6231201295284 [s⁻¹]
y: 132.07160469079943 [s⁻¹]
z: 154.60252075588042 [s⁻¹]


In [13]:
print(f'Equilibrium Emittances:')
print(f'From Radiation Integrals:')
print(f'x: {tw.rad_int_eq_gemitt_x} [m rad]')
print(f'y: {tw.rad_int_eq_gemitt_y} [m rad]\n')

print(f'From Twiss:')
print(f'x: {tw.eq_gemitt_x} [m rad]')
print(f'y: {tw.eq_gemitt_y} [m rad]')
print(f'z: {tw.eq_gemitt_zeta} [m rad]')

Equilibrium Emittances:
From Radiation Integrals:
x: 3.8320302877932464e-10 [m rad]
y: 1.6125738813963252e-49 [m rad]

From Twiss:
x: 1.6635987304922333e-10 [m rad]
y: 1.8398217115203367e-51 [m rad]
z: 1.0006871405270933e-06 [m rad]
